In [1]:
import pandas as pd
import geopandas as gpd
import os
import yaml

In [2]:
os.chdir('..')

In [3]:
from pilates.utils import geog

In [4]:
blocks = gpd.read_file('/home/mgardner/austin_blocks.shp')
del blocks['TAZ']

In [6]:
taz = gpd.read_file('/home/mgardner/campo_zoning.shp')

In [8]:
with open('settings.yaml') as file:
    settings = yaml.load(file, Loader=yaml.FullLoader)

In [9]:
local_crs = settings['local_crs']['austin']

In [21]:
block_to_taz = geog.get_taz_from_block_geoms(blocks, taz, local_crs, 'zone').astype(int)

In [22]:
b2 = blocks.merge(block_to_taz.reset_index(), on=['GEOID'])

In [23]:
b2[['GEOID','zone']].to_csv('pilates/polaris/data/austin_blocks_to_taz.csv',index=False)

In [24]:
b2['zone'].nunique()

2093